In [1]:
import arrow
import pandas as pd
import bs4 as bs
import requests
import icalendar

pd.set_option('max_colwidth', 0)

In [3]:
base_url = 'https://tnb.liga.nu'
season = 'Sommer+2020'
teams = [2467046, 2467113]
for team in teams:
    url = f'{base_url}/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/teamPortrait?team={team}&championship=TNB+{season}'
    [df] = pd.read_html(url, match='Mannschaft', parse_dates=True, index_col=0)
    [portrait] = df.to_dict().values()

    print(portrait['Mannschaft'], portrait['Liga'], sep=': ')

    soup = bs.BeautifulSoup(requests.get(url).content, 'lxml')
    tables = soup.find_all('table')
    if len(tables) > 3:
        table = tables[3]

        data = []
        links = []
        for row in table.find_all('tr'):
            row_data = []
            row_links = []
            for td in row.find_all('td'):
                row_data.append(''.join(td.stripped_strings))
                if td.find('a'):
                    row_links.append(base_url + td.a['href'])
            if row_data:
                data.append(row_data)
            if row_links:
                links.append(row_links)

        column_names = ['Tag', 'Datum'] + [th.text.strip() for th in table.find_all('th')][1:] or None
        if 'Heimmannschaft' not in column_names:
            print(column_names)
        else:
            df = pd.DataFrame(data, columns=column_names)
            # display(df)
            df_links = pd.DataFrame(links, columns=['team_link', 'meeting_link'][:len(links[0])])
            # display(df_links)
            t = pd.concat([df.iloc[:,1:4], df_links], axis='columns')
            display(t)

            cal = icalendar.Calendar(method='REQUEST')
            for row in t.itertuples(index=False):
                try:
                    begin = arrow.get(row.Datum, 'DD.MM.YYYY HH:mm', tzinfo='Europe/Berlin')
                except arrow.parser.ParserError:
                    begin = arrow.get(row.Datum, 'DD.MM.YYYY', tzinfo='Europe/Berlin').replace(hour=10, minute=1)
                event = icalendar.Event(
                )
                event.add('dtstart', begin.datetime)
                event.add('dtstamp', arrow.now().datetime)
                event.add('uid', f'{portrait["Liga"]}: {row.Heimmannschaft} vs. {row.Gastmannschaft}')
                event.add('summary',  portrait["Liga"])
                description = [f'{row.Heimmannschaft} vs. {row.Gastmannschaft}',
                               f'<a href={url}>{portrait["Liga"]}</a>',
                               f'<a href={row.team_link}>Gegnermannschaft</a>']
                if hasattr(row, 'meeting_link'):
                    description.append(f'<a href={row.meeting_link}>Spielbericht</a>')
                event.add('description', '<br>'.join(description))
                event.add('dtstart', begin.datetime)
                event.add('dtend', begin.shift(hours=6).datetime)
                event.add('location', row.Heimmannschaft)
                event.add('organizer', 'TNB')

                cal.add_component(event)
            with open(f'{portrait["Liga"]}.ics', 'wb') as f:
                f.write(cal.to_ical())
            print(f'"{portrait["Liga"]}.ics"')
            #print(cal.to_ical().decode())
    else:
        print(tables[-1])

TC Alfeld: Herren 40 Regionsliga Gr. 409
"Herren 40 Regionsliga Gr. 409.ics"
TC Alfeld: Damen 1. Regionsklasse Gr. 039
"Damen 1. Regionsklasse Gr. 039.ics"


,Datum,Heimmannschaft,Gastmannschaft,team_link
0,03.05.2020,TC GRE Hildesheim II,TC Alfeld,https://tnb.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/teamPortrait?federation=TNB&region=DE.NO.TNB&team=2463658
1,10.05.2020,TC Alfeld,Bortfelder TC e.V. von 1977,https://tnb.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/teamPortrait?federation=TNB&region=DE.NO.TNB&team=2463303
2,17.05.2020,TC Alfeld,TSV Salzgitter Abt. Tennis,https://tnb.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/teamPortrait?federation=TNB&region=DE.NO.TNB&team=2465131
3,24.05.2020,MTV E Borsum,TC Alfeld,https://tnb.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/teamPortrait?federation=TNB&region=DE.NO.TNB&team=2465444
4,21.06.2020,TC Alfeld,SV Anker 06 e.V. Gadenstedt II,https://tnb.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/teamPortrait?federation=TNB&region=DE.NO.TNB&team=2468418
5,12.07.2020,TV Eintracht Algermissen II,TC Alfeld,https://tnb.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/teamPortrait?federation=TNB&region=DE.NO.TNB&team=2467822


,Datum,Heimmannschaft,Gastmannschaft,team_link
0,03.05.2020,TC Alfeld,TSV Holtensen,https://tnb.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/teamPortrait?federation=TNB&region=DE.NO.TNB&team=2466751
1,10.05.2020,TC Alfeld,MTV Rosdorf von 1909 e. V.,https://tnb.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/teamPortrait?federation=TNB&region=DE.NO.TNB&team=2464609
2,24.05.2020,TV Blau-Weiß Einbeck e.V.,TC Alfeld,https://tnb.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/teamPortrait?federation=TNB&region=DE.NO.TNB&team=2466718
3,07.06.2020,TC Alfeld,SCW Göttingen III,https://tnb.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/teamPortrait?federation=TNB&region=DE.NO.TNB&team=2462751
4,21.06.2020,MTV Almstedt II,TC Alfeld,https://tnb.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/teamPortrait?federation=TNB&region=DE.NO.TNB&team=2465295
5,12.07.2020,TV Rot-Weiss Bad Salzdetfurth,TC Alfeld,https://tnb.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/teamPortrait?federation=TNB&region=DE.NO.TNB&team=2468113
